# Working with Text Data


Follow _Introduction to Machine Learning_ [Chapter 7](https://github.com/amueller/introduction_to_ml_with_python/blob/master/07-working-with-text-data.ipynb):
-  Chapter 7. Working with text data - 7.7 Bag of Words with more than one word (p.329-350)


We will look at supervised classification of text features, in particular sentiment analysis. Given a text commenting on a movie, is this a *positive* or a *negative* comment.

Similar problems include for example, given an email text, is this *spam* or *legitimate* message.

Our classifiers will only accept numerical features as input. Therefore, we have to **transform** text to a **numerical representation** first.

We will start by looking at pre-processing techniques using toy data in this lecture.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import mglearn

## Bag of words
Computing the bag-of-words representation for a **corpus** of **documents** consists of the following three steps:

1. Tokenization. Split each document into the words that appear in it (called tokens), for example by splitting them on whitespace and punctuation.

2. Vocabulary building. Collect a vocabulary of all words that appear in any of the documents, and number them (say, in alphabetical order).

3. Encoding. For each document, count how often each of the words in the vocabulary appear in this document.

In [3]:
bards_words =["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

`bards_words` is a corpus.

`"The fool doth think he is wise,"` is a document.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer()

In [5]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("Vocabulary content:\n {}".format(vect.vocabulary_))

Vocabulary size: 13
Vocabulary content:
 {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [6]:
bag_of_words = vect.transform(bards_words)
print("bag_of_words: {}".format(repr(bag_of_words)))

bag_of_words: <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [7]:
print("Dense representation of bag_of_words:\n{}".format(
    bag_of_words.toarray()))

Dense representation of bag_of_words:
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


In [8]:
vect.get_feature_names_out()

array(['be', 'but', 'doth', 'fool', 'he', 'himself', 'is', 'knows', 'man',
       'the', 'think', 'to', 'wise'], dtype=object)

In [9]:
pd.DataFrame(bag_of_words.toarray(), columns=vect.get_feature_names_out())

,be,but,doth,fool,he,himself,is,knows,man,the,think,to,wise
0,0,0,1,1,1,0,1,0,0,1,1,0,1
1,1,1,0,1,0,1,0,1,1,1,0,1,1


Looking at the tokenization process.

In [10]:
tokenizer = vect.build_analyzer()
print(bards_words[0])
print(tokenizer(bards_words[0]))

The fool doth think he is wise,
['the', 'fool', 'doth', 'think', 'he', 'is', 'wise']


In [11]:
tokenizer("I am sure that this is   an #awesome hyper-text at https:\\ucalgary.ca. ")

['am',
 'sure',
 'that',
 'this',
 'is',
 'an',
 'awesome',
 'hyper',
 'text',
 'at',
 'https',
 'ucalgary',
 'ca']

### Another example with repeating words

In [12]:
repeating_words = ["The sun, the sun, shines so bright, so bright.",
                   "The moon, the moon, reflects so bright, so bright."]
vect = CountVectorizer()
vect.fit(repeating_words)
print(vect.vocabulary_)
bag_of_words = vect.transform(repeating_words)
pd.DataFrame(bag_of_words.toarray(), columns=vect.get_feature_names_out())

{'the': 6, 'sun': 5, 'shines': 3, 'so': 4, 'bright': 0, 'moon': 1, 'reflects': 2}


,bright,moon,reflects,shines,so,sun,the
0,2,0,0,1,2,2,2
1,2,2,1,0,2,0,2


## Improving Bag-of-words: min_df
> One way to cut back on these is to only use tokens that appear in at least two documents (or at least five documents, and so on). A token that appears only in a single document is unlikely to appear in the test set and is therefore not helpful. We can set the minimum number of documents a token needs to appear in with the min_df parameter:

In [13]:
vect = CountVectorizer(min_df=2)
vect.fit(bards_words)
bag_of_words = vect.transform(bards_words)
pd.DataFrame(bag_of_words.toarray(), columns=vect.get_feature_names_out())

,fool,the,wise
0,1,1,1
1,1,1,1


## Improving Bag-of-words: removing stopwords
>Another way that we can get rid of uninformative words is by discarding words that are too frequent to be informative. There are two main approaches: using a language-specific list of stopwords, or discarding words that appear too frequently. scikit-learn has a built-in list of English stopwords in the feature_extraction.text module

In [14]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print("Number of stop words: {}".format(len(ENGLISH_STOP_WORDS)))
print("Every 10th stopword:\n{}".format(list(ENGLISH_STOP_WORDS)[::10]))

Number of stop words: 318
Every 10th stopword:
['couldnt', 'someone', 'onto', 'seems', 'whom', 'noone', 'toward', 'cannot', 'behind', 'too', 'ie', 'between', 'him', 'if', 'us', 'an', 'show', 'somewhere', 'further', 'me', 'i', 'many', 'cant', 'within', 'the', 'give', 'becoming', 'a', 'is', 'have', 'therefore', 'during']


In [15]:
vect = CountVectorizer(stop_words="english")
vect.fit(bards_words)
bag_of_words = vect.transform(bards_words)
pd.DataFrame(bag_of_words.toarray(), columns=vect.get_feature_names_out())

,doth,fool,knows,man,think,wise
0,1,1,0,0,1,1
1,0,1,1,1,0,1


## Tf-idf
[Sklearn Doc - TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

>Instead of dropping features that are deemed unimportant, another approach is to rescale features by how informative we expect them to be. One of the most common ways to do this is using the term frequency–inverse document frequency (tf–idf) method. 

>The intuition of this method is to give high weight to any term that appears often in a particular document, but not in many documents in the corpus. 

>If a word appears often in a particular document, but not in very many documents, it is likely to be very descriptive of the content of that document.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
vect.fit(bards_words)

TfidfVectorizer()

In [17]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("Vocabulary content:\n {}".format(vect.vocabulary_))

Vocabulary size: 13
Vocabulary content:
 {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [18]:
tfidf_words = vect.transform(bards_words)
print("tfidf_words: {}".format(repr(tfidf_words)))

tfidf_words: <2x13 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [19]:
print("Dense representation of tfidf_words:\n{}".format(
    tfidf_words.toarray()))

Dense representation of tfidf_words:
[[0.         0.         0.42567716 0.30287281 0.42567716 0.
  0.42567716 0.         0.         0.30287281 0.42567716 0.
  0.30287281]
 [0.36469323 0.36469323 0.         0.25948224 0.         0.36469323
  0.         0.36469323 0.36469323 0.25948224 0.         0.36469323
  0.25948224]]


In [20]:
tfidf_df = pd.DataFrame(tfidf_words.toarray(), columns=vect.get_feature_names_out())
tfidf_df

,be,but,doth,fool,he,himself,is,knows,man,the,think,to,wise
0,0.000000,0.000000,0.425677,0.302873,0.425677,0.000000,0.425677,0.000000,0.000000,0.302873,0.425677,0.000000,0.302873
1,0.364693,0.364693,0.000000,0.259482,0.000000,0.364693,0.000000,0.364693,0.364693,0.259482,0.000000,0.364693,0.259482


### Another example with repeating words

In [21]:
repeating_words = ["The sun, the sun, shines so bright, so bright.",
                   "The moon, the moon, reflects so bright, so bright."]
vect = TfidfVectorizer()
vect.fit(repeating_words)
tfidf_words = vect.transform(repeating_words)
pd.DataFrame(tfidf_words.toarray(), columns=vect.get_feature_names_out())

,bright,moon,reflects,shines,so,sun,the
0,0.427602,0.000000,0.00000,0.30049,0.427602,0.600979,0.427602
1,0.427602,0.600979,0.30049,0.00000,0.427602,0.000000,0.427602


### Without stop-words

In [22]:
repeating_words = ["The sun, the sun, shines so bright, so bright.",
                   "The moon, the moon, reflects so bright, so bright."]
vect = TfidfVectorizer(stop_words="english")
vect.fit(repeating_words)
tfidf_words = vect.transform(repeating_words)
pd.DataFrame(tfidf_words.toarray(), columns=vect.get_feature_names_out())

,bright,moon,reflects,shines,sun
0,0.536893,0.000000,0.000000,0.377292,0.754584
1,0.536893,0.754584,0.377292,0.000000,0.000000


## Bag-of-Words with More Than One Word (n-Grams)
>One of the main disadvantages of using a bag-of-words representation is that word order is completely discarded. Therefore, the two strings “it’s bad, not good at all” and “it’s good, not bad at all” have exactly the same representation, even though the meanings are inverted. Putting “not” in front of a word is only one example (if an extreme one) of how context matters. 

>Fortunately, there is a way of capturing context when using a bag-of-words representation, by not only considering the counts of single tokens, but also the counts of pairs or triplets of tokens that appear next to each other. 

>Pairs of tokens are known as bigrams, triplets of tokens are known as trigrams, and more generally sequences of tokens are known as n-grams. 

>We can change the range of tokens that are considered as features by changing the ngram_range parameter of CountVectorizer or TfidfVectorizer.

In [23]:
bards_words

['The fool doth think he is wise,',
 'but the wise man knows himself to be a fool']

In [24]:
cv = CountVectorizer(ngram_range=(1, 1)).fit(bards_words)
print("Vocabulary size: {}".format(len(cv.vocabulary_)))
print("Vocabulary:\n{}".format(cv.get_feature_names_out()))

Vocabulary size: 13
Vocabulary:
['be' 'but' 'doth' 'fool' 'he' 'himself' 'is' 'knows' 'man' 'the' 'think'
 'to' 'wise']


In [25]:
cv = CountVectorizer(ngram_range=(2, 2)).fit(bards_words)
print("Vocabulary size: {}".format(len(cv.vocabulary_)))
print("Vocabulary:\n{}".format(cv.get_feature_names_out()))

Vocabulary size: 14
Vocabulary:
['be fool' 'but the' 'doth think' 'fool doth' 'he is' 'himself to'
 'is wise' 'knows himself' 'man knows' 'the fool' 'the wise' 'think he'
 'to be' 'wise man']


In [26]:
cv = CountVectorizer(ngram_range=(1, 2)).fit(bards_words)
print("Vocabulary size: {}".format(len(cv.vocabulary_)))
print("Vocabulary:\n{}".format(cv.get_feature_names_out()))

Vocabulary size: 27
Vocabulary:
['be' 'be fool' 'but' 'but the' 'doth' 'doth think' 'fool' 'fool doth'
 'he' 'he is' 'himself' 'himself to' 'is' 'is wise' 'knows'
 'knows himself' 'man' 'man knows' 'the' 'the fool' 'the wise' 'think'
 'think he' 'to' 'to be' 'wise' 'wise man']


### Another example with repeating words

In [27]:
repeating_words = ["The sun, the sun, shines so bright, so bright.",
                   "The moon, the moon, reflects so bright, so bright."]
cv = CountVectorizer(ngram_range=(1, 2)).fit(repeating_words)
print("Vocabulary size: {}".format(len(cv.vocabulary_)))
print("Vocabulary:\n{}".format(cv.get_feature_names_out()))

Vocabulary size: 17
Vocabulary:
['bright' 'bright so' 'moon' 'moon reflects' 'moon the' 'reflects'
 'reflects so' 'shines' 'shines so' 'so' 'so bright' 'sun' 'sun shines'
 'sun the' 'the' 'the moon' 'the sun']


In [28]:
ngram_words = cv.transform(repeating_words)
pd.DataFrame(ngram_words.toarray(), columns=cv.get_feature_names_out())

,bright,bright so,moon,moon reflects,moon the,reflects,reflects so,shines,shines so,so,so bright,sun,sun shines,sun the,the,the moon,the sun
0,2,1,0,0,0,0,0,1,1,2,2,2,1,1,2,0,2
1,2,1,2,1,1,1,1,0,0,2,2,0,0,0,2,2,0


### Td-idf n-gram

In [29]:
repeating_words = ["The sun, the sun, shines so bright, so bright.",
                   "The moon, the moon, reflects so bright, so bright."]
vect = TfidfVectorizer(ngram_range=(1,2))
vect.fit(repeating_words)
tfidf_words = vect.transform(repeating_words)
pd.DataFrame(tfidf_words.toarray(), columns=vect.get_feature_names_out())

,bright,bright so,moon,moon reflects,moon the,reflects,reflects so,shines,shines so,so,so bright,sun,sun shines,sun the,the,the moon,the sun
0,0.313481,0.156741,0.000000,0.000000,0.000000,0.000000,0.000000,0.220293,0.220293,0.313481,0.313481,0.440587,0.220293,0.220293,0.313481,0.000000,0.440587
1,0.313481,0.156741,0.440587,0.220293,0.220293,0.220293,0.220293,0.000000,0.000000,0.313481,0.313481,0.000000,0.000000,0.000000,0.313481,0.440587,0.000000


### Td-idf n-gram no stop-words

In [30]:
repeating_words = ["The sun, the sun, shines so bright, so bright.",
                   "The moon, the moon, reflects so bright, so bright."]
vect = TfidfVectorizer(ngram_range=(1,2), stop_words="english")
vect.fit(repeating_words)
tfidf_words = vect.transform(repeating_words)
pd.DataFrame(tfidf_words.toarray(), columns=vect.get_feature_names_out())

,bright,bright bright,moon,moon moon,moon reflects,reflects,reflects bright,shines,shines bright,sun,sun shines,sun sun
0,0.438501,0.21925,0.000000,0.000000,0.000000,0.000000,0.000000,0.308149,0.308149,0.616298,0.308149,0.308149
1,0.438501,0.21925,0.616298,0.308149,0.308149,0.308149,0.308149,0.000000,0.000000,0.000000,0.000000,0.000000
